<a href="https://colab.research.google.com/github/Flymajt/Web-project/blob/main/JupyterNotebook/SQL/Where_limit_groupBy_logFunkce_zadani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WHERE, LIMIT, GROUP BY, logické funkce
Všechny tyto funkce se používají k bližší specifikaci toho co chceme z databáze vypsat, používají se tedy v rámci dotazu SELECT.

Zatímco za slovo SELECT zapisujeme které SLOUPEČKY chceme z tabulky získat, za klíčové slovo WHERE zapisujeme co musí splňovat ŘÁDKY které se mají ve výsledku zobrazit.

Dále také za WHERE zapisujeme, zda mají být řádky seřazené, seskupené a pod.

<code>SELECT sloupec1, sloupec2 FROM Nazev_tabulky WHERE omezení_řádků další_nastavení </code>


In [1]:
! pip install mysql.connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 31.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp311-cp311-linux_x86_64.whl size=247949 sha256=a35e9f70b6fdc5dbe67554efffa6ea7d14820fc39b6252a9c034447fb1b0fa15
  Stored in directory: /root/.cache/pip/wheels/17/cd/ed/2d49e9bac69cf09382e4c7cc20a2511202b48324b87db26019
Successfully built mysql.connector


## LIMIT
Omezuje počet vypsaných záznamů. Jinými slovy zobrazí prvních X záznamů.

Použití může vypadat následovně.

In [8]:
import mysql.connector

mydb = mysql.connector.connect(
    host="dbs.spskladno.cz",
    user="student24",
    password="spsnet",
    database="vyuka24",
)
mycursor = mydb.cursor()

# Dříve, než můžeme z tabulky vypisovat musíme ji vytvořít a zapsat do ní.

# Kontrola jestli tabulka neexistuje, pokud by existovala, smažeme ji.
try:
    mycursor.execute("""DROP TABLE Automobil;""")
    mydb.commit()
except mysql.connector.Error:
    print("Tato tabulka neexistuje.")

# Vytvoříme tabulku Automobil
mycursor.execute("""CREATE TABLE Automobil(
    id int,
    spz text,
    pocet_sedadel int,
    max_rychlost int,
    nosnost int,
    nutna_kvalifikace text,
    datum_vyroby date
)""")

mydb.commit()

# Zapíšeme do ni
mycursor.execute("""INSERT INTO Automobil
(id, spz, pocet_sedadel, max_rychlost, nosnost,  nutna_kvalifikace, datum_vyroby) VALUES
(1, '1A1111', 4, 190, 3, 'B',  '2000-09-09'),
(2, '2B2222', 2, 220, 2, 'B',  '2020-01-01');
""")

mydb.commit()

mycursor.execute("""SELECT pocet_sedadel, max_rychlost FROM Automobil LIMIT 1
""") # Tento příkaz vypíše počet sedadel a maximální rychlost, ale pouze jeden záznam.

myresult = mycursor.fetchall()

for pocet_sedadel, max_rychlost in myresult:
    print(f"""
    Auto má {pocet_sedadel} sedadel a maximální rychlost {max_rychlost} km/h.
    """)


mycursor.close()
mydb.close()


    Auto má 4 sedadel a maximální rychlost 190 km/h.
    


## GROUP BY
Sloučí duplicitní hodnoty v daném atributu. Tedy pokud se v jednom atributu nachází 2 a více stejných hodnot, tak se sloučí a vypíší se pouze jednou.

Použití může vypadat následovně.

In [9]:
import mysql.connector

mydb = mysql.connector.connect(
    host="dbs.spskladno.cz",
    user="student24",
    password="spsnet",
    database="vyuka24",
)
mycursor = mydb.cursor()

# Vypůčíme si taulku z předchozí ukázky
# a dopíšeme do ní nějaké záznamy s duplicitními hodnotami

mycursor.execute("""INSERT INTO Automobil
(id, spz, pocet_sedadel, max_rychlost, nosnost,  nutna_kvalifikace, datum_vyroby) VALUES
(3, '1A1111', 5, 180, 5, 'D',  '2002-09-09'),
(4, '3C3333', 2, 220, 2, 'B',  '2020-05-01');
""")

mydb.commit()

# Nyní použijeme GROUP BY na vypsání unikátních SPZ.
mycursor.execute("""SELECT spz FROM Automobil GROUP BY spz
""")

myresult = mycursor.fetchall()

for spz in myresult:
    print(f"""
    SPZ: {spz}
    """)

mycursor.close()
mydb.close()


    SPZ: ('1A1111',)
    

    SPZ: ('2B2222',)
    

    SPZ: ('3C3333',)
    


## WHERE
Kllíčové slovo, které nám dovolí zapsat jakoukoliv podmínku, kterou musí řádek splňovat, aby se zobrazil.

Použití může vypadat následovně.

In [10]:
import mysql.connector

mydb = mysql.connector.connect(
    host="dbs.spskladno.cz",
    user="student24",
    password="spsnet",
    database="vyuka24",
)
mycursor = mydb.cursor()

# Vypůčíme si taulku z předchozích ukázek
# a vypíšeme z ní Automobil, který má ID = 3.

mycursor.execute("""SELECT * FROM Automobil WHERE id = 3
""")

myresult = mycursor.fetchall()

for index, spz, pocet_sedadel, max_rychlost, nosnost, nutna_kvalifikace, datum_vyroby in myresult:
    print(f"""
    Auto s ID {index}, SPZ {spz}, počtem sedadel {pocet_sedadel},
    maximální rychlostí {max_rychlost}, nosností {nosnost},
    nutná kvalifikace pro řízení {nutna_kvalifikace}, vyrobeno {datum_vyroby}
    """)

print("======")
# nebo ty, která mají počet sedadel více než 3
mycursor.execute("""SELECT * FROM Automobil WHERE pocet_sedadel > 3
""")

myresult = mycursor.fetchall()

for index, spz, pocet_sedadel, max_rychlost, nosnost, nutna_kvalifikace, datum_vyroby in myresult:
    print(f"""
    Auto s ID {index}, SPZ {spz}, počtem sedadel {pocet_sedadel},
    maximální rychlostí {max_rychlost}, nosností {nosnost},
    nutná kvalifikace pro řízení {nutna_kvalifikace}, vyrobeno {datum_vyroby}
    """)

mycursor.close()
mydb.close()


    Auto s ID 3, SPZ 1A1111, počtem sedadel 5, 
    maximální rychlostí 180, nosností 5, 
    nutná kvalifikace pro řízení D, vyrobeno 2002-09-09
    

    Auto s ID 1, SPZ 1A1111, počtem sedadel 4, 
    maximální rychlostí 190, nosností 3, 
    nutná kvalifikace pro řízení B, vyrobeno 2000-09-09
    

    Auto s ID 3, SPZ 1A1111, počtem sedadel 5, 
    maximální rychlostí 180, nosností 5, 
    nutná kvalifikace pro řízení D, vyrobeno 2002-09-09
    


## Logické funkce
SŘBD umí logické funkce AND, OR, NOT. Tyto funkce se používají spolu s WHERE.

AND - vrátí záznamy z tabulky, které splňují všechny podmínky

OR - vrátí záznamy z tabulky, které splňují alespoň jednu podmínku

NOT - vrátí záznamy z tabulky, které nesplňují podmínku

SQL kód pro tyto logické funkce může vypadat následovně

In [11]:
import mysql.connector

mydb = mysql.connector.connect(
    host="dbs.spskladno.cz",
    user="student24",
    password="spsnet",
    database="vyuka24",
)
mycursor = mydb.cursor()


# Použití funkce AND
# Vypůčíme si taulku Automobi z předchozích ukázek
# a vypíšeme z ní Automobil, který má spz = 1A1111 a id = 1.
mycursor.execute("""SELECT * FROM Automobil WHERE spz = '1A1111' AND id = 1
""")
myresult = mycursor.fetchall()


for index, spz, pocet_sedadel, max_rychlost, nosnost, nutna_kvalifikace, datum_vyroby in myresult:
    print(f"""
    Auto s ID {index}, SPZ {spz}, počtem sedadel {pocet_sedadel},
    maximální rychlostí {max_rychlost}, nosností {nosnost},
    nutná kvalifikace pro řízení {nutna_kvalifikace}, vyrobeno {datum_vyroby}
    """)
print("====")


# Použití funkce OR
# Nyní vypíšeme řádky, které mají pocet_sedadel = 2 nebo nutna_kvalifikace = 'D'
mycursor.execute("""SELECT * FROM Automobil WHERE
pocet_sedadel = 2 OR nutna_kvalifikace = 'D'
""")

myresult = mycursor.fetchall()

for index, spz, pocet_sedadel, max_rychlost, nosnost, nutna_kvalifikace, datum_vyroby in myresult:
    print(f"""
    Auto s ID {index}, SPZ {spz}, počtem sedadel {pocet_sedadel},
    maximální rychlostí {max_rychlost}, nosností {nosnost},
    nutná kvalifikace pro řízení {nutna_kvalifikace}, vyrobeno {datum_vyroby}
    """)
print("====")

# Použití funkce NOT
# Vypíšeme řádky, které namají id = 3

mycursor.execute("""SELECT * FROM Automobil WHERE NOT id = 3
""")

myresult = mycursor.fetchall()

for index, spz, pocet_sedadel, max_rychlost, nosnost, nutna_kvalifikace, datum_vyroby in myresult:
    print(f"""
    Auto s ID {index}, SPZ {spz}, počtem sedadel {pocet_sedadel},
    maximální rychlostí {max_rychlost}, nosností {nosnost},
    nutná kvalifikace pro řízení {nutna_kvalifikace}, vyrobeno {datum_vyroby}
    """)

mycursor.close()
mydb.close()


    Auto s ID 1, SPZ 1A1111, počtem sedadel 4, 
    maximální rychlostí 190, nosností 3, 
    nutná kvalifikace pro řízení B, vyrobeno 2000-09-09
    
====

    Auto s ID 2, SPZ 2B2222, počtem sedadel 2, 
    maximální rychlostí 220, nosností 2, 
    nutná kvalifikace pro řízení B, vyrobeno 2020-01-01
    

    Auto s ID 3, SPZ 1A1111, počtem sedadel 5, 
    maximální rychlostí 180, nosností 5, 
    nutná kvalifikace pro řízení D, vyrobeno 2002-09-09
    

    Auto s ID 4, SPZ 3C3333, počtem sedadel 2, 
    maximální rychlostí 220, nosností 2, 
    nutná kvalifikace pro řízení B, vyrobeno 2020-05-01
    
====

    Auto s ID 1, SPZ 1A1111, počtem sedadel 4, 
    maximální rychlostí 190, nosností 3, 
    nutná kvalifikace pro řízení B, vyrobeno 2000-09-09
    

    Auto s ID 2, SPZ 2B2222, počtem sedadel 2, 
    maximální rychlostí 220, nosností 2, 
    nutná kvalifikace pro řízení B, vyrobeno 2020-01-01
    

    Auto s ID 4, SPZ 3C3333, počtem sedadel 2, 
    maximální rychlostí 220, 

## Cvičení
Bude následovat pár úkolů na procvičení

### Cvičení 1:
Přihlašte se do školní databáze a vypište z tabulky Automobil 3 záznamy.

In [15]:
import mysql.connector

mydb = mysql.connector.connect(
    host="dbs.spskladno.cz",
    user="student24",
    password="spsnet",
    database="vyuka24",
)

mycursor = mydb.cursor()
try:
  mycursor.execute("""SELECT * FROM Automobil LIMIT 3
  """)
  myresult = mycursor.fetchall()
  print(myresult)
  pass

except mysql.connector.Error as chyba:
    print("Příkaz byl zadán chybně:\n", chyba)

mycursor.close()
mydb.close()

[(1, '1A1111', 4, 190, 3, 'B', datetime.date(2000, 9, 9)), (2, '2B2222', 2, 220, 2, 'B', datetime.date(2020, 1, 1)), (3, '1A1111', 5, 180, 5, 'D', datetime.date(2002, 9, 9))]


### Referenční řešení pro kontrolu:
<pre>
Auto s ID 1, SPZ 1A1111, počtem sedadel 4,
maximální rychlostí 190, nosností 3,
nutná kvalifikace pro řízení B, vyrobeno 2000-09-09

Auto s ID 2, SPZ 2B2222, počtem sedadel 2,
maximální rychlostí 220, nosností 2,
nutná kvalifikace pro řízení B, vyrobeno 2020-01-01

Auto s ID 3, SPZ 1A1111, počtem sedadel 5,
maximální rychlostí 180, nosností 5,
nutná kvalifikace pro řízení D, vyrobeno 2002-09-09
</pre>


### Cvičení 2:
Přihlašte se do školní databáze a vypište z tabulky Automobil pouze unikátní záznamy atributu pocet_sedadel.

In [22]:
import mysql.connector

mydb = mysql.connector.connect(
    host="dbs.spskladno.cz",
    user="student24",
    password="spsnet",
    database="vyuka24",
)
mycursor = mydb.cursor()

try:
  mycursor.execute("""SELECT DISTINCT pocet_sedadel FROM Automobil
  """)
  myresult = mycursor.fetchall()
  print(myresult)
  pass

except mysql.connector.Error as chyba:
    print("Příkaz byl zadán chybně:\n", chyba)

mycursor.close()
mydb.close()

[(4,), (2,), (5,)]


### Referenční řešení pro kontrolu:
<pre>
počet sedadel: (2,)
počet sedadel: (4,)
počet sedadel: (5,)
</pre>


### Cvičení 3:
Přihlašte se do školní databáze a vypište z tabulky Automobil záznamy, které mají atribut datum_vyroby = '2002-09-09'.

In [23]:
import mysql.connector

mydb = mysql.connector.connect(
    host="dbs.spskladno.cz",
    user="student24",
    password="spsnet",
    database="vyuka24",
)
mycursor = mydb.cursor()

try:
  mycursor.execute("""SELECT * FROM Automobil WHERE datum_vyroby = '2002-09-09'
  """)
  myresult = mycursor.fetchall()
  print(myresult)
  pass

except mysql.connector.Error as chyba:
    print("Příkaz byl zadán chybně:\n", chyba)

mycursor.close()
mydb.close()

[(3, '1A1111', 5, 180, 5, 'D', datetime.date(2002, 9, 9))]


### Referenční řešení pro kontrolu:
<pre>
Auto s ID 3, SPZ 1A1111, počtem sedadel 5,
maximální rychlostí 180, nosností 5,
nutná kvalifikace pro řízení D, vyrobeno 2002-09-09
</pre>

### Cvičení 4:
Přihlašte se do školní databáze a vypište z tabulky Automobil
* záznamy, které mají pocet_sedadel = 2 nebo spz = '1A1111'. Vypište pouze atributy pocet_sedadel a spz.

* Poté vypište záznamy, které mají spz = '1A1111' a max_rychlost = 180. Vypište pouze atributy spz a max_rychlost.

* Nakonec vypište záznamy, které nemají nutna_kvalifikace = 'D'. Vypište pouze atrbuty id a nutna_kvalifikace.

In [29]:
import mysql.connector

mydb = mysql.connector.connect(
    host="dbs.spskladno.cz",
    user="student24",
    password="spsnet",
    database="vyuka24",
)
mycursor = mydb.cursor()

try:
  mycursor.execute("""SELECT pocet_sedadel, spz FROM Automobil WHERE pocet_sedadel = 2 or spz = '1A1111'
  """)
  myresult = mycursor.fetchall()

  mycursor.execute("""SELECT spz, max_rychlost FROM Automobil WHERE spz = '1A1111 and max_rychlost = 180'
  """)
  myresult2 = mycursor.fetchall()

  mycursor.execute("""SELECT id, nutna_kvalifikace FROM Automobil WHERE NOT nutna_kvalifikace = 'D'
  """)
  myresult3 = mycursor.fetchall()

  print(myresult, myresult2, myresult3)
  pass



except mysql.connector.Error as chyba:
    print("Příkaz byl zadán chybně:\n", chyba)

mycursor.close()
mydb.close()

[(4, '1A1111'), (2, '2B2222'), (5, '1A1111'), (2, '3C3333')] [] [(1, 'B'), (2, 'B'), (4, 'B')]


### Referenční řešení pro kontrolu:
<pre>
Auto s SPZ 1A1111 má 4 sedadel
Auto s SPZ 2B2222 má 2 sedadel
Auto s SPZ 1A1111 má 5 sedadel
Auto s SPZ 3C3333 má 2 sedadel
====
Auto s SPZ 1A1111 jede až 180
====
Auto s ID 1, můžete řídit, jen když máte kvalifikaci B
Auto s ID 2, můžete řídit, jen když máte kvalifikaci B
Auto s ID 4, můžete řídit, jen když máte kvalifikaci B
</pre>